# Usando variáveis numéricas e categóricas juntas

Nos cadernos anteriores, mostramos o pré-processamento necessário para aplicar
ao lidar com variáveis numéricas e categóricas. No entanto, nós dissociamos
o processo para tratar cada tipo individualmente. Neste caderno, vamos mostrar
como combinar essas etapas de pré-processamento.

Primeiro, carregaremos todo o conjunto de dados do censo de adultos.

In [1]:
import pandas as pd

adult_census = pd.read_csv("adult-census.csv")
# drop the duplicated column `"education-num"` as stated in the first notebook
adult_census = adult_census.drop(columns=['ID','fnlwgt:','education-num:'])

target_name = "class"
target = adult_census[target_name]

data = adult_census.drop(columns=target_name)

adult_census.head()

,age,workclass,education:,marital-status:,occupation:,relationship:,race:,sex:,capital-gain:,capital-loss:,hours-per-week:,native-country:,class
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## Seleção baseada em tipos de dados

Separaremos variáveis categóricas e numéricas usando seus dados
tipos para identificá-los, como vimos anteriormente que `objeto` corresponde
para colunas categóricas (strings). Fazemos uso de `make_column_selector`
auxiliar para selecionar as colunas correspondentes.

In [2]:
from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(data)
categorical_columns = categorical_columns_selector(data)

## Despachar colunas para um processador específico

Nas seções anteriores, vimos que precisamos tratar os dados de maneira diferente
dependendo de sua natureza (ou seja, numérica ou categórica).

Scikit-learn fornece uma classe `ColumnTransformer` que enviará
colunas para um transformador específico, tornando mais fácil ajustar um único preditivo
modelo em um conjunto de dados que combina os dois tipos de variáveis juntos
(dados tabulares digitados de forma heterogênea).

Primeiro definimos as colunas dependendo de seu tipo de dados:

* **codificação one-hot** será aplicada a colunas categóricas. Além disso, nós
  use `handle_unknown =" ignore "` para resolver os problemas potenciais devido ao raro
  categorias.
* **escala numérica** recursos numéricos que serão padronizados.

Agora, criamos nosso `ColumnTransfomer` especificando três valores:
o nome do pré-processador, o transformador e as colunas.
Primeiro, vamos criar os pré-processadores para os dados numéricos e categóricos
partes.

In [3]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

Agora, criamos o transformador e associamos cada um desses pré-processadores
com suas respectivas colunas.

In [4]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard-scaler', numerical_preprocessor, numerical_columns)])

Podemos levar um minuto para representar graficamente a estrutura de um
`ColumnTransformer`:


![Predictor fit diagram](imagens/column_trans.png)


Um `ColumnTransformer` faz o seguinte:

* Ele **divide as colunas** do conjunto de dados original com base nos nomes das colunas
  ou índices fornecidos. Obteremos tantos subconjuntos quanto o número de
  transformadores passados para o `ColumnTransformer`.
* Ele **transforma cada subconjunto**. Um transformador específico é aplicado a
  cada subconjunto: ele chamará internamente `fit_transform` ou` transform`. O
  a saída desta etapa é um conjunto de conjuntos de dados transformados.
* Em seguida, ** concatena os conjuntos de dados transformados ** em um único conjunto de dados.

O importante é que `ColumnTransformer` é como qualquer outro
transformador scikit-learn. Em particular, pode ser combinado com um classificador
em um `Pipeline`:

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

model = make_pipeline(preprocessor, LogisticRegression(max_iter=500))

Podemos exibir um diagrama interativo com o seguinte comando:

In [6]:
from sklearn import set_config
set_config(display='diagram')
model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['workclass', 'education:',
                                                   'marital-status:',
                                                   'occupation:',
                                                   'relationship:', 'race:',
                                                   'sex:', 'native-country:']),
                                                 ('standard-scaler',
                                                  StandardScaler(),
                                                  ['age', 'capital-gain:',
                                                   'capital-loss:',
                                                   'hours-per-week:'])])),
                ('logisticregression', LogisticRegression(max_iter=500))])

O modelo final é mais complexo do que os modelos anteriores, mas ainda segue
a mesma API (o mesmo conjunto de métodos que pode ser chamado pelo usuário):

- o método `fit` é chamado para pré-processar os dados e, em seguida, treinar o
  classificador dos dados pré-processados;
- o método `predict` faz previsões sobre novos dados;
- o método `score` é usado para prever os dados de teste e comparar o
  previsões para os rótulos de teste esperados para calcular a precisão.

Vamos começar dividindo nossos dados em conjuntos de treinamento e teste.

In [7]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=42)

<div class="admonition caution alert alert-warning">
<p class="first admonition-title" style="font-weight: bold;">Caution!</p>
<p class="last">Esteja ciente de que usamos<tt class="docutils literal">train_test_split</tt> aqui para fins didáticos, para mostrar
a API scikit-learn.</p>
</div>

Agora, podemos treinar o modelo no conjunto de trem.

In [8]:
_ = model.fit(data_train, target_train)

Então, podemos enviar o conjunto de dados bruto direto para o pipeline. Na verdade, nós não
precisa fazer qualquer pré-processamento manual (chamando o `transform` ou
métodos `fit_transform`), pois será tratado ao chamar o` predict`
método. Como exemplo, prevemos nas cinco primeiras amostras do teste
definir.

In [9]:
data_test.head()

,age,workclass,education:,marital-status:,occupation:,relationship:,race:,sex:,capital-gain:,capital-loss:,hours-per-week:,native-country:
14160,27,Private,Some-college,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,38,United-States
27048,45,State-gov,HS-grad,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States
28868,29,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,Black,Male,0,0,55,United-States
5667,30,Private,Bachelors,Never-married,Machine-op-inspct,Not-in-family,White,Female,0,0,40,United-States
7827,29,Self-emp-not-inc,Some-college,Divorced,Craft-repair,Not-in-family,White,Male,2202,0,50,United-States


In [10]:
model.predict(data_test)[:5]

array(['<=50K', '<=50K', '>50K', '<=50K', '<=50K'], dtype=object)

In [11]:
target_test[:5]

14160    <=50K
27048    <=50K
28868     >50K
5667     <=50K
7827     <=50K
Name: class, dtype: object

Para obter diretamente a pontuação de precisão, precisamos chamar o método `score`. Vamos
calcule a pontuação de precisão em todo o conjunto de teste.

In [12]:
model.score(data_test, target_test)

0.8578798673381648

## Avaliação do modelo com validação cruzada

Como afirmado anteriormente, um modelo preditivo deve ser avaliado por
validação cruzada. Nosso modelo pode ser usado com as ferramentas de validação cruzada de
scikit-learn como qualquer outro preditor:

In [13]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, data, target, cv=5)
cv_results

{'fit_time': array([1.41780925, 0.87463403, 1.01599312, 1.66795349, 1.31983948]),
 'score_time': array([0.057796  , 0.02890229, 0.02682996, 0.04101801, 0.06900096]),
 'test_score': array([0.84861047, 0.84735872, 0.85273342, 0.85657248, 0.85304054])}

In [14]:
scores = cv_results["test_score"]
print("A mean cross-validation accuracy é: "
      f"{scores.mean():.3f} +/- {scores.std():.3f}")

A mean cross-validation accuracy é: 0.852 +/- 0.003


O modelo composto tem uma maior precisão preditiva do que os dois modelos que
utilizou variáveis numéricas e categóricas isoladamente.

## Adaptando um modelo mais poderoso

**Linear models** são bons porque geralmente são baratos para treinar,
**pequeno** para implantar, **rápido** para prever e fornecer uma **boa linha de base**.

No entanto, muitas vezes é útil verificar se modelos mais complexos, como um
conjunto de árvores de decisão pode levar a um desempenho preditivo mais alto. Nisso
seção, usaremos um modelo chamado **gradient-boosting trees** e
avaliar seu desempenho estatístico. Mais precisamente, o modelo scikit-learn
usaremos é chamado de `HistGradientBoostingClassifier`. Observe que impulsionar
os modelos serão abordados com mais detalhes em um módulo futuro.

Para modelos baseados em árvore, o tratamento de variáveis numéricas e categóricas é
mais simples do que para modelos lineares:
* nós **não precisamos dimensionar os recursos numéricos**
* usar uma **codificação ordinal para as variáveis categóricas** é bom, mesmo se
  a codificação resulta em uma ordem arbitrária

Portanto, para `HistGradientBoostingClassifier`, o pipeline de pré-processamento
é um pouco mais simples do que o que vimos anteriormente para `LogisticRegression`:

In [18]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import OrdinalEncoder

#categorical_preprocessor = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
categorical_preprocessor = OrdinalEncoder()
preprocessor = ColumnTransformer([
    ('categorical', categorical_preprocessor, categorical_columns)],
    remainder="passthrough")

model = make_pipeline(preprocessor, HistGradientBoostingClassifier())

Agora que criamos nosso modelo, podemos verificar seu desempenho estatístico.

In [19]:
%%time
_ = model.fit(data_train, target_train)

Wall time: 2.14 s


In [20]:
model.score(data_test, target_test)

0.8760594521557549

Podemos observar que obtemos precisões significativamente maiores com o Gradiente
Modelo de reforço. Isso é frequentemente o que observamos sempre que o conjunto de dados tem um
grande número de amostras e número limitado de recursos informativos (por exemplo, menos
de 1000) com uma mistura de variáveis numéricas e categóricas.

Isso explica por que as máquinas com gradiente aumentado são muito populares entre
profissionais de ciência de dados que trabalham com dados tabulares.

Neste caderno nós:

* usou um `ColumnTransformer` para aplicar diferentes pré-processamento para
  variáveis categóricas e numéricas;
* usou um pipeline para encadear o pré-processamento `ColumnTransformer` e
  ajuste de regressão logística;
* visto que **gradient boosting methods** podem superar o desempenho **linear
  models**.

# 📝 Exercício M1.05

O objetivo deste exercício é avaliar o impacto do pré-processamento de recursos
em um pipeline que usa um classificador baseado em árvore de decisão em vez de logístico
regressão.

- A primeira questão é avaliar empiricamente se a escala numérica
  recurso é útil ou não;
- A segunda questão é avaliar se é empiricamente melhor (ambos
  de uma perspectiva computacional e estatística) para usar códigos inteiros ou
  categorias codificadas one-hot.

In [1]:
import pandas as pd

adult_census = pd.read_csv("adult-census.csv")
# drop the duplicated column `"education-num"` as stated in the first notebook
adult_census = adult_census.drop(columns=['ID','fnlwgt:','education-num:'])

target_name = "class"
target = adult_census[target_name]

data = adult_census.drop(columns=target_name)

adult_census.head()

,age,workclass,education:,marital-status:,occupation:,relationship:,race:,sex:,capital-gain:,capital-loss:,hours-per-week:,native-country:,class
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Como nos blocos de notas anteriores, usamos o utilitário `make_column_selector`
para selecionar apenas a coluna com um tipo de dados específico. Além disso, nós listamos em
avance todas as categorias para as colunas categóricas.

In [2]:
from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)
numerical_columns = numerical_columns_selector(data)
categorical_columns = categorical_columns_selector(data)

## Pipeline de referência (sem escala numérica e categorias codificadas por inteiro)

Primeiro vamos cronometrar o pipeline que usamos no notebook principal para servir como um
referência:

In [ ]:
%%time
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

#categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value",unknown_value=-1)
categorical_preprocessor = OrdinalEncoder()

preprocessor = ColumnTransformer([
    ('categorical', categorical_preprocessor, categorical_columns)],
    remainder="passthrough")

model = make_pipeline(preprocessor, HistGradientBoostingClassifier())
cv_results = cross_validate(model, data, target)
scores = cv_results["test_score"]
print("The mean cross-validation accuracy is: "
      f"{scores.mean():.3f} +/- {scores.std():.3f}")

## Dimensionando recursos numéricos

Vamos escrever um pipeline semelhante que também dimensiona os recursos numéricos usando
`StandardScaler` (ou semelhante):

In [ ]:
%%time
from sklearn.preprocessing import StandardScaler

preprocessor = ColumnTransformer([
    ('numerical', StandardScaler(), numerical_columns),
    ('categorical', OrdinalEncoder(),
     categorical_columns)])

model = make_pipeline(preprocessor, HistGradientBoostingClassifier())
cv_results = cross_validate(model, data, target)
scores = cv_results["test_score"]
print("The mean cross-validation accuracy is: "
      f"{scores.mean():.3f} +/- {scores.std():.3f}")

## Codificação one-hot de variáveis categóricas

Para modelos lineares, observamos que a codificação inteira de
variáveis podem ser muito prejudiciais. Porém para
Modelos `HistGradientBoostingClassifier`, não parece ser o caso, como
a validação cruzada do pipeline de referência com `OrdinalEncoder` é boa.

Vamos ver se podemos obter uma precisão ainda melhor com `OneHotEncoder`.

Dica: `HistGradientBoostingClassifier` ainda não suporta entrada esparsa
dados. Você pode querer usar
`OneHotEncoder (handle_unknown =" ignore ", sparse = False)` para forçar o uso de um
representação densa como uma solução alternativa.

In [7]:
%%time
from sklearn.preprocessing import OneHotEncoder

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore", sparse=False)

preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns)],
    remainder="passthrough")

model = make_pipeline(preprocessor, HistGradientBoostingClassifier())
cv_results = cross_validate(model, data, target)
scores = cv_results["test_score"]
print("The mean cross-validation accuracy is: "
      f"{scores.mean():.3f} +/- {scores.std():.3f}")

The mean cross-validation accuracy is: 0.873 +/- 0.003
Wall time: 30.8 s
